In [1]:
import nltk
import random
import json
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Download necessary NLTK resources
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [16]:
# Load intents from JSON file
with open('intents.json', 'r') as f:
    intents = json.load(f)

# Initialize stemmer
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenize(sentence):
    """Split sentence into array of words/tokens."""
    return nltk.word_tokenize(sentence)

def stem(word):
    """Find the root form of the word."""
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, words):
    """Return bag of words array."""
    sentence_words = [stem(word) for word in tokenized_sentence]
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words:
            bag[idx] = 1
    return bag

In [17]:
# Prepare training data
all_words = []
tags = []
xy = []

for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

# Stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

In [18]:
# Create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)


In [43]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out) # Pass 'out' to l2 instead of 'x'
        out = self.relu(out)
        out = self.l3(out)
        return out

In [44]:
# Hyper-parameters
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)


In [45]:
class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [46]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)

        outputs = model(words)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

print(f'Final loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 0.9445
Epoch [200/1000], Loss: 0.0441
Epoch [300/1000], Loss: 0.0070
Epoch [400/1000], Loss: 0.0029
Epoch [500/1000], Loss: 0.0021
Epoch [600/1000], Loss: 0.0013
Epoch [700/1000], Loss: 0.0004
Epoch [800/1000], Loss: 0.0006
Epoch [900/1000], Loss: 0.0002
Epoch [1000/1000], Loss: 0.0001
Final loss: 0.0001


In [47]:
data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "all_words": all_words,
    "tags": tags
}

FILE = "chatbot_model.pth"
torch.save(data, FILE)

print(f'Training complete. File saved to {FILE}')


Training complete. File saved to chatbot_model.pth


In [48]:
# Context and Entity-aware Chatbot
class ChatBot:
    def __init__(self):
        self.context = None
        self.entities = {}

    def get_response(self, user_input):
        """Get a response based on user input, context, and entities."""
        sentence = tokenize(user_input)
        X = bag_of_words(sentence, all_words)
        X = torch.from_numpy(X).to(device)
        output = model(X)
        _, predicted = torch.max(output, dim=0)
        tag = tags[predicted.item()]

        # Find the intent
        for intent in intents['intents']:
            if tag == intent['tag']:
                # Update context and entities
                self.context = random.choice(intent.get('context', [None]))
                for entity in intent.get('entities', []):
                    if entity in sentence:
                        self.entities[entity] = True

                # Return appropriate response
                response = random.choice(intent['responses'])

                # Append entity information if relevant
                if self.entities:
                    response += f" (Detected entities: {', '.join(self.entities.keys())})"
                return response

        return "I'm sorry, I don't understand."

In [49]:
# Create chatbot instance and start conversation
chatbot = ChatBot()

print("Let's chat! (type 'quit' to exit)")
while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
        break
    response = chatbot.get_response(user_input)
    print(f"Bot: {response}")

Let's chat! (type 'quit' to exit)
You: quit


In [50]:
def validate_model(test_data, model, all_words, tags):
    """
    Validate the chatbot model on a test dataset.

    Parameters:
        test_data (list): A list of tuples where each tuple contains a user input and its expected tag.
                          Example: [("How can I apply?", "admissions"), ...]
        model (NeuralNet): The trained chatbot model.
        all_words (list): The vocabulary used for training.
        tags (list): The list of intent tags.

    Returns:
        float: Accuracy of the model on the test dataset.
    """
    model.eval()  # Set the model to evaluation mode
    correct = 0

    for user_input, expected_tag in test_data:
        # Preprocess user input
        tokenized_input = tokenize(user_input)
        bag = bag_of_words(tokenized_input, all_words)
        bag_tensor = torch.from_numpy(bag).unsqueeze(0).to(device)  # Add batch dimension and move to device

        # Get the model prediction
        output = model(bag_tensor)
        _, predicted_idx = torch.max(output, dim=1)
        predicted_tag = tags[predicted_idx.item()]

        # Compare with the expected tag
        if predicted_tag == expected_tag:
            correct += 1

    accuracy = correct / len(test_data) * 100
    return accuracy

# Example Test Data
test_data = [
    # Admissions-related questions
    ("How can I apply for admission?", "admissions"),
    ("What are the admission requirements?", "admissions"),
    ("When is the application deadline?", "admissions"),
    ("Can I apply online?", "admissions"),
    ("Is the admission process open now?", "admissions"),
    ("Where can I find the admission form?", "admissions"),
    ("What documents are needed for admission?", "admissions"),
    ("Do you have rolling admissions?", "admissions"),

    # Courses-related questions
    ("Can I see the list of courses?", "courses"),
    ("What courses are offered?", "courses"),
    ("Do you have engineering programs?", "courses"),
    ("What are the popular courses?", "courses"),
    ("Can you tell me about business programs?", "courses"),
    ("Do you offer data science programs?", "courses"),
    ("Are there any evening courses?", "courses"),
    ("Is there a course in AI?", "courses"),

    # Fees-related questions
    ("How much does the MBA program cost?", "fees"),
    ("What is the tuition fee?", "fees"),
    ("Are there any scholarships available?", "fees"),
    ("What is the fee structure?", "fees"),
    ("Is there any financial aid for students?", "fees"),
    ("Do international students pay more fees?", "fees"),
    ("Can I pay fees in installments?", "fees"),
    ("What is the cost for a bachelor's program?", "fees"),

    # Transportation-related questions
    ("Do you provide transportation facilities?", "transportation"),
    ("Is there a bus service for students?", "transportation"),
    ("What are the transport charges?", "transportation"),
    ("Do you offer transportation on weekends?", "transportation"),
    ("Can I get a bus schedule?", "transportation"),
    ("How far does the campus transportation go?", "transportation"),
    ("Is the transportation service reliable?", "transportation"),

    # Events-related questions
    ("Tell me about campus events.", "events"),
    ("Are there any upcoming cultural events?", "events"),
    ("What events are happening this month?", "events"),
    ("Do you host any workshops?", "events"),
    ("When is the next seminar?", "events"),
    ("What activities are planned for freshmen?", "events"),
    ("Is there a sports fest?", "events"),

    # Placement-related questions
    ("What are the placement statistics?", "placement"),
    ("Do you have a placement cell?", "placement"),
    ("Which companies visit for placements?", "placement"),
    ("What is the placement process?", "placement"),
    ("Can I get placement support as an alumni?", "placement"),
    ("What is the average package for engineering students?", "placement"),
    ("Do startups hire from your campus?", "placement"),

    # Facilities-related questions
    ("Is there a library on campus?", "facilities"),
    ("What facilities are available?", "facilities"),
    ("Is there a sports complex?", "facilities"),
    ("Can you tell me about hostel facilities?", "facilities"),
    ("Do you have a gym on campus?", "facilities"),
    ("Are there labs for research students?", "facilities"),
    ("What amenities are there for international students?", "facilities"),

    # Library-related questions
    ("What are the library timings?", "library"),
    ("Can I borrow books from the library?", "library"),
    ("Do you have an online library?", "library"),
    ("Are there study rooms in the library?", "library"),
    ("Does the library provide access to research papers?", "library"),
    ("Can I use the library as an external member?", "library"),

    # Scholarships-related questions
    ("Are there any scholarships available?", "scholarships"),
    ("How can I apply for scholarships?", "scholarships"),
    ("Do you offer merit-based scholarships?", "scholarships"),
    ("What are the eligibility criteria for scholarships?", "scholarships"),
    ("Are there scholarships for international students?", "scholarships"),
    ("Can I apply for multiple scholarships?", "scholarships"),
    ("Is financial aid available for low-income families?", "scholarships"),

    # Hostel-related questions
    ("What are the hostel charges?", "hostel"),
    ("Do you have separate hostels for boys and girls?", "hostel"),
    ("What amenities are provided in the hostel?", "hostel"),
    ("Is there 24/7 security in the hostel?", "hostel"),
    ("Can I get a single room in the hostel?", "hostel"),
    ("Do hostels have Wi-Fi?", "hostel"),
    ("Are meals included in the hostel fee?", "hostel"),
]


# Validate the model
accuracy = validate_model(test_data, model, all_words, tags)
print(f"Model Accuracy: {accuracy:.2f}%")


Model Accuracy: 63.89%
